In [117]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup("""
<?xml version="1.0" encoding="UTF-8"?>
<!-- generated by CLiX/Wiki2XML [MPI-Inf, MMCI@UdS] $LastChangedRevision: 92 $ on 16.04.2009 15:20:19[mciao0826] -->
<!DOCTYPE article SYSTEM "../article.dtd">
<article xmlns:xlink="http://www.w3.org/1999/xlink">
<bdy>

In <link xlink:type="simple" xlink:href="../831/18831.xml">mathematics</link>

<sec>
machin, bidule
</sec>
</bdy>
</article>
""", "xml")

In [6]:
soup

<?xml version="1.0" encoding="utf-8"?>
<?xml version="1.0" encoding="UTF-8"?><!-- generated by CLiX/Wiki2XML [MPI-Inf, MMCI@UdS] $LastChangedRevision: 92 $ on 16.04.2009 15:20:19[mciao0826] --><!DOCTYPE article SYSTEM "../article.dtd">
<article xmlns:xlink="http://www.w3.org/1999/xlink">
<bdy>

In <link xlink:href="../831/18831.xml" xlink:type="simple">mathematics</link>
<sec>
machin, bidule
</sec>
</bdy>
</article>

In [5]:
s = "/article[1]/bdy[1]/sec[2]"

i = -1
try:
    while True:
        i = s.index("/", i + 1)
        print(i, s[:i])
except ValueError:
    print(i, s)


0 
11 /article[1]
18 /article[1]/bdy[1]
18 /article[1]/bdy[1]/sec[2]


In [94]:
from typing import Generator, Tuple, Dict, TypeVar, Any, List
import os
import os.path as path

T = TypeVar('T')
K = TypeVar('K')
V = TypeVar('V')

In [31]:
RUN_RESULT_EXT = ".i.txt"

def get_runs(d: str) -> Generator[Tuple[str, str], None, None]:
    for f in os.listdir(d):
        if f.endswith(RUN_RESULT_EXT):
            id = f[:-len(RUN_RESULT_EXT)]
            run_file = id + ".txt"
            yield id, path.join(d, f), path.join(d, run_file)


In [38]:
def get_or_create_subdic(dic: Dict[T, Dict[K, V]], key: T) -> Dict[K, V]:
    if key in dic:
        return dic[key]
    dic[key] = dict()
    return dic[key]

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [110]:
class RunFile:
    def __init__(self, id: str, magp: float) -> None:
        self.magp = magp
        rd = id.split("_")
        self.id = id[len(rd[0]) + 2 + len(rd[1]):]
        self.grp = rd[0]
        self.algo = rd[2]
        self.mode = rd[3]
        self.stop = rd[4]
        self.stem = rd[5]

        self.options = []
        for i in range(6, len(rd)):
            self.options.append(rd[i])




In [111]:
def get_runs_data(dir: str) -> List[Dict[str, Any]]:

    runs = []

    for rid, run_result, run_file in get_runs(dir):
        with open(run_result) as fr:
            lines = fr.readlines()

        top = 1 + 7 * 21

        magp = 0.0

        for i in range(top, top + 33):
            id, _, value = lines[i].split()

            if id == 'MAgP':
                magp = float(value)
        try:
            runs.append(RunFile(rid, magp))
        except:
            pass

    runs.sort(key=lambda d: d.magp, reverse=True)

    return runs



In [112]:
def as_df(dir: str):
    return pd.DataFrame(map(lambda r: {
        # "id": r.id,
        "grp": r.grp,
        "c_algo": r.algo,
        "c_mode": r.mode,
        "c_stop": r.stop,
        "c_stem": r.stem,
        "c_cfg": str(r.options),
        "magp": r.magp
    }, get_runs_data(dir)))


In [116]:
as_df("runs/runs2_res")

,grp,c_algo,c_mode,c_stop,c_stem,c_cfg,magp
0,RemiClementSantiagoJulie,bm25,articles,stop174,stemmer,"['k1.2', 'b0.6']",0.209015
1,RemiClementSantiagoJulie,bm25,articles,stop174,stemmer,"['k1.2', 'b0.5']",0.208107
2,RemiClementSantiagoJulie,bm25,articles,stop174,stemmer,"['k1.2', 'b0.4']",0.207664
3,RemiClementSantiagoJulie,bm25,articles,stop174,stemmer,"['k1.2', 'b0.7']",0.206372
4,RemiClementSantiagoJulie,bm25,articles,stop174,nostem,"['k1.2', 'b0.75']",0.205198
...,...,...,...,...,...,...,...
217,TheoAurelienMaximeAlexandre,BM25,docno,nostop,nostem,"['k1.5', 'b0.90']",0.002942
218,TheoAurelienMaximeAlexandre,BM25,docno,stop,stem,"['k1.428', 'b0.828']",0.002908
219,TheoAurelienMaximeAlexandre,BM25,docno,nostop,stem,"['k1', 'b0.8']",0.002405
220,TheoAurelienMaximeAlexandre,BM25,docno,nostop,stem,"['k1.44', 'b0.85']",0.002340
